In [2]:
# Data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Reading data
df = pd.read_csv('C:/Users/isabe/Documents/player_weekly_data.csv')
df.head()

,Name,Team,Pos,Wk,Opp,Year,Status,TeamScore,OppScore,PassYds,...,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgs,ReceivingYds,ReceivingTD,FL
0,Aaron Jones,GB,RB,2,DET,2020,0,42,21,0,...,0,0,18,168,2,4,8,68,1,0
1,Dak Prescott,DAL,QB,2,ATL,2020,0,40,39,450,...,47,34,5,18,3,0,0,0,0,1
2,Alvin Kamara,NO,RB,2,LV,2020,0,24,34,0,...,0,0,13,79,2,9,9,95,0,0
3,Cam Newton,NE,QB,2,SEA,2020,0,30,35,397,...,44,30,11,47,2,0,0,0,0,0
4,Josh Allen,BUF,QB,2,MIA,2020,0,31,28,415,...,35,24,4,19,0,0,0,0,0,0


In [ ]:
# Removing NaN 
df = df.dropna()

In [12]:
# Top 10 Players with the most Passing TD in one week
top_10_assists = df.sort_values(by='PassingTD', ascending=False).head(10)
top_10_assists.head(10)

,Name,Team,Pos,Wk,Opp,Year,Status,TeamScore,OppScore,PassYds,...,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgs,ReceivingYds,ReceivingTD,FL
5,Russell Wilson,SEA,QB,2,NE,2020,0,35,30,288,...,28,21,5,39,0,0,0,0,0,0
4,Josh Allen,BUF,QB,2,MIA,2020,0,31,28,415,...,35,24,4,19,0,0,0,0,0,0
9,Matt Ryan,ATL,QB,2,DAL,2020,0,39,40,273,...,36,24,4,16,0,0,0,0,0,0
15,Ryan Tannehill,TEN,QB,2,JAX,2020,0,33,30,239,...,24,18,4,12,0,0,0,0,0,0
35,Derek Carr,LV,QB,2,NO,2020,0,34,24,284,...,38,28,3,3,0,0,0,0,0,2
20,Joe Burrow,CIN,QB,2,CLE,2020,0,30,35,316,...,61,37,7,19,0,0,0,0,0,2
26,Jared Goff,LAR,QB,2,PHI,2020,0,37,19,267,...,27,20,7,13,0,0,0,0,0,0
27,Gardner Minshew II,JAX,QB,2,TEN,2020,0,30,33,339,...,45,30,4,19,0,0,0,0,0,1
83,Baker Mayfield,CLE,QB,2,CIN,2020,0,35,30,219,...,23,16,3,5,0,0,0,0,0,0
110,Mitchell Trubisky,CHI,QB,2,NYG,2020,0,17,13,190,...,28,18,4,16,0,0,0,0,0,0


In [17]:
# Top 10 Players with the most Passing TD (all weeks combined)
top_10_assists_overall = df.groupby('Name')['PassingTD'].sum()
top_10_assists_overall = top_10_assists_overall.sort_values(ascending=False).head(10)
top_10_assists_overall

Name
Russell Wilson        5
Matt Ryan             4
Josh Allen            4
Ryan Tannehill        4
Gardner Minshew II    3
Derek Carr            3
Joe Burrow            3
Jared Goff            3
Mitchell Trubisky     2
Baker Mayfield        2
Name: PassingTD, dtype: int64

In [22]:
# Teams with the most pass completion 
teams_completion = df.groupby('Team').Cmp.sum()
teams_completion = teams_completion.sort_values(ascending=False)
teams_completion

Team
CIN    37
DAL    34
CAR    33
MIA    31
NE     30
JAX    30
PIT    29
LV     28
KC     27
ARI    26
NO     26
PHI    26
NYG    25
HOU    25
ATL    24
BUF    24
TB     23
LAC    22
SF     22
SEA    21
NYJ    21
DET    20
LAR    20
DEN    19
IND    19
WAS    19
GB     18
CHI    18
TEN    18
BAL    18
CLE    16
MIN    11
Name: Cmp, dtype: int64

In [5]:
# AVGS by Team 
df.groupby('Team').mean()  

,Wk,Year,TeamScore,OppScore,PassYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgs,ReceivingYds,ReceivingTD,FL
Team,,,,,,,,,,,,,,,,,
ARI,2.0,2020.0,30.0,15.0,7.150000,0.025000,0.000000,0.950000,0.650000,0.800000,4.000000,0.050000,0.650000,0.850000,7.150000,0.025000,0.025000
ATL,2.0,2020.0,39.0,40.0,7.000000,0.102564,0.000000,0.948718,0.615385,0.871795,2.897436,0.000000,0.615385,0.923077,7.000000,0.102564,0.000000
BAL,2.0,2020.0,33.0,16.0,5.230769,0.025641,0.051282,0.615385,0.461538,0.948718,5.897436,0.025641,0.461538,0.589744,5.230769,0.025641,0.000000
BUF,2.0,2020.0,31.0,28.0,10.641026,0.102564,0.000000,0.897436,0.615385,0.589744,2.871795,0.000000,0.615385,0.897436,10.641026,0.102564,0.025641
CAR,2.0,2020.0,17.0,31.0,9.410256,0.000000,0.051282,1.076923,0.846154,0.615385,2.230769,0.051282,0.846154,1.025641,9.410256,0.000000,0.076923
CHI,2.0,2020.0,17.0,13.0,4.871795,0.051282,0.051282,0.717949,0.461538,0.820513,3.461538,0.000000,0.435897,0.692308,4.769231,0.051282,0.025641
CIN,2.0,2020.0,30.0,35.0,8.315789,0.078947,0.052632,1.605263,0.973684,0.631579,1.789474,0.000000,0.973684,1.552632,8.315789,0.078947,0.052632
CLE,2.0,2020.0,35.0,30.0,5.763158,0.052632,0.000000,0.605263,0.421053,0.921053,5.657895,0.078947,0.421053,0.552632,5.763158,0.052632,0.000000
DAL,2.0,2020.0,40.0,39.0,11.538462,0.025641,0.000000,1.230769,0.871795,0.794872,3.205128,0.102564,0.871795,1.205128,11.538462,0.025641,0.102564
